In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016

@author: Hossam Faris
"""

import random
import numpy
import math
# from solution import solution
import time
from bioproc.proc_opt import BioProc
from bioproc.proc_models import *


def GWO(objf, lb, ub, dim, SearchAgents_no, Max_iter):
    print("I am in GWO")

    # Max_iter=1000
    # lb=-100
    # ub=100
    # dim=30
    # SearchAgents_no=5

    # initialize alpha, beta, and delta_pos
    Alpha_pos = numpy.zeros(dim)
    Alpha_score = float("inf")

    Beta_pos = numpy.zeros(dim)
    Beta_score = float("inf")

    Delta_pos = numpy.zeros(dim)
    Delta_score = float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim

    # Initialize the positions of search agents
    Positions = numpy.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = (
            numpy.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
        )

    Convergence_curve = numpy.zeros(Max_iter)
    # s = solution()

    # Loop counter
    print('GWO is optimizing  "' + objf.__name__ + '"')

    timerStart = time.time()
    # s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0, Max_iter):
        for i in range(0, SearchAgents_no):

            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i, j] = numpy.clip(Positions[i, j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness = objf(Positions[i, :])

            # Update Alpha, Beta, and Delta
            if fitness[0] < Alpha_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = Alpha_score  # Update beta
                Beta_pos = Alpha_pos.copy()
                Alpha_score = fitness[0]
                # Update alpha
                Alpha_pos = Positions[i, :].copy()

            if fitness[0] > Alpha_score and fitness[0] < Beta_score:
                Delta_score = Beta_score  # Update delte
                Delta_pos = Beta_pos.copy()
                Beta_score = fitness[0]  # Update beta
                Beta_pos = Positions[i, :].copy()

            if fitness[0] > Alpha_score and fitness[0] > Beta_score and fitness[0] < Delta_score:
                Delta_score = fitness[0]  # Update delta
                Delta_pos = Positions[i, :].copy()

        a = 2 - l * ((2) / Max_iter)
        # a decreases linearly fron 2 to 0

        # Update the Position of search agents including omegas
        for i in range(0, SearchAgents_no):
            for j in range(0, dim):

                r1 = random.random()  # r1 is a random number in [0,1]
                r2 = random.random()  # r2 is a random number in [0,1]

                A1 = 2 * a * r1 - a
                # Equation (3.3)
                C1 = 2 * r2
                # Equation (3.4)

                D_alpha = abs(C1 * Alpha_pos[j] - Positions[i, j])
                # Equation (3.5)-part 1
                X1 = Alpha_pos[j] - A1 * D_alpha
                # Equation (3.6)-part 1

                r1 = random.random()
                r2 = random.random()

                A2 = 2 * a * r1 - a
                # Equation (3.3)
                C2 = 2 * r2
                # Equation (3.4)

                D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 2
                X2 = Beta_pos[j] - A2 * D_beta
                # Equation (3.6)-part 2

                r1 = random.random()
                r2 = random.random()

                A3 = 2 * a * r1 - a
                # Equation (3.3)
                C3 = 2 * r2
                # Equation (3.4)

                D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
                # Equation (3.5)-part 3
                X3 = Delta_pos[j] - A3 * D_delta
                # Equation (3.5)-part 3

                Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)

        Convergence_curve[l] = Alpha_score

        if l % 1 == 0:
            print(
                ["At iteration " + str(l) + " the best fitness is " + str(Alpha_score)]
            )

    timerEnd = time.time()
    # s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    # s.executionTime = timerEnd - timerStart
    # s.convergence = Convergence_curve
    # s.optimizer = "GWO"
    # s.objfname = objf.__name__

    return Convergence_curve

In [7]:
def model_neg_f(point, model): # negated, so the optimization problem is a minimization problem!
    return -model.eval(np.array([point]).T)[0]
point = numpy.array([4]*12)

model_mode = four_bit_processor_ext


param_values = {"transcription": {"min": 0.01, "max": 50},
                "translation": {"min": 0.01, "max": 50},
                "protein_production": {"min": 0.1, "max": 50},
                "rna_degradation": {"min": 0.1, "max": 100},
                "protein_degradation": {"min": 0.001, "max": 50},
                "hill": {"min": 1, "max": 5},
                "Kd": {"min": 0.01, "max": 250},
                "protease_concentration": {"min": 10, "max": 1000}
                }

model = BioProc(np.array(
    ["protein_production", "protein_production", "protein_production", "protein_production", "protein_degradation",
     "protein_degradation", "Kd", "hill", "protein_production", "protein_degradation", "Kd", "hill"]),
                model_mode=model_mode, parameter_values=param_values, avg_dev=30)
lb = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.01, 1, 0.1, 0.1, 0.01, 1]
ub = [50, 50, 50, 50, 50, 50, 250, 5, 50, 50, 250, 5]
GWO(model.eval, lb, ub, 12, 10, 10)

-17
I am in GWO
GWO is optimizing  "eval"
This is cunt: 1
-99.92668481751491
This is best so far I guess: -99.92668481751491
This is cunt: 2
-99.926684734002
This is best so far I guess: -99.926684734002
This is cunt: 3
-99.9266836875743
This is best so far I guess: -99.9266836875743
This is cunt: 4
-99.92668481751488
This is cunt: 5
-99.92666529631737
This is best so far I guess: -99.92666529631737
This is cunt: 6
-99.92667319314165
This is cunt: 7
-99.92668470693536
This is cunt: 8
-99.92668481751153
This is cunt: 9
-99.92668288370767
This is cunt: 10
-99.9266840375311
['At iteration 0 the best fitness is -99.92668481751491']
This is cunt: 11
-99.92668481751491
This is cunt: 12
-99.92668481751491
This is cunt: 13
-99.92668481046813
This is cunt: 14
-99.92668479666338
This is cunt: 15
-99.92668481759429
This is cunt: 16
-99.92668481749678
This is cunt: 17
-99.9266848175148
This is cunt: 18
-99.92668481901973
This is cunt: 19
-99.92668481751491
This is cunt: 20
-99.92668481751501
['At 

array([-99.92668482, -99.92668482, -99.92668482, -99.92669259,
       -99.92669259, -99.92669933, -99.92669933, -99.92673898,
       -99.92675093, -99.92677973])